### Imports

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import sys
import json
import os
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from terminaltables import AsciiTable
from collections import Counter
from nltk.corpus import wordnet as wn
from collections import defaultdict

from nltk.corpus import stopwords
stop_words_en = set(stopwords.words('english'))

PATH_TO_DATA = '../data'

article_db = None
reduced_article_db = None

os.makedirs(PATH_TO_DATA, exist_ok=True)

/usr/local/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### Load reduced dataset

In [2]:
# Load reduced article db
if os.path.exists('%s/articles' % PATH_TO_DATA):
    print ("Loading reduced article DB...")
    reduced_article_db = pd.read_pickle('%s/articles' % PATH_TO_DATA)
    print ("+ Done.")

Loading reduced article DB...
+ Done.


### Load full dataset

In [3]:
signalmedia_url = 'http://research.signalmedia.co/newsir16/signal-dataset.html'

# Assert that dataset file is downloaded and in 'data/'
assert os.path.exists('%s/signalmedia-1m.jsonl' % PATH_TO_DATA), \
        "Please download SignalMedia dataset from %s and save it to 'data/'\
         before executing this cell." % signalmedia_url

# Load full article DB
lines = []
with open('%s/signalmedia-1m.jsonl' % PATH_TO_DATA, 'r') as f:
    for json_article in tqdm(f.readlines(), desc='Loading articles'):
        article = dict(json.loads(json_article))
        # Extract article id, content and media type
        lines.append({'id': article['id'], 
                      'content': article['content'], 
                      'media-type': article['media-type']})

print ("Load as Pandas DF...")
article_db = pd.DataFrame(lines)
print ("+ Done.")

# One-hot encoding of 'media-type': 'News' = 0, 'Blog = 1
article_db.loc[article_db['media-type'] == 'News', 'blog'] = 0
article_db.loc[article_db['media-type'] == 'Blog', 'blog'] = 1

# Remove rows that has NaN values
article_db.dropna(inplace=True)

Loading articles: 100%|██████████| 1000000/1000000 [01:07<00:00, 14827.26it/s]


Load as Pandas DF...
+ Done.


### Display column names

In [4]:
assert article_db is not None or reduced_article_db is not None, \
    "Load either the full article dataset or the reduced article \
    dataset before executing this cell."

# Extract column names
cols = article_db.columns if article_db is not None else reduced_article_db.columns

# Show column names
print ("Columns:")
for col in cols:
    print ("+ %s" % col)

Columns:
+ content
+ id
+ media-type
+ blog


### Reduce and balance dataset

In [5]:
def print_media_occ(article_type, articles):
    """Count number of occurences of each class and print as a table"""
    print (article_type)
    num_news = np.sum(articles['blog'] == 0)
    num_blogs = np.sum(articles['blog'] == 1)
    
    table_data = [["Media Type", "Count"]]
    for media_type, count in [("News", num_news), ("Blog", num_blogs)]:
        table_data.append([media_type, count])
    print (AsciiTable(table_data).table)
    
if article_db is not None:
    # Display media type occurences of the original dataset
    print_media_occ('Article DB', article_db)
    print ()

if reduced_article_db is None and article_db is not None:
    print ("Reduce and balance dataset...")
    n_of_each = 10000
    news_articles = article_db[article_db['blog'] == 0].iloc[:n_of_each]
    blog_articles = article_db[article_db['blog'] == 1].iloc[:n_of_each]
    reduced_article_db = pd.concat([news_articles, blog_articles])
    print ("+ Done.\n")
        
    reduced_article_db.to_pickle("data/articles")
    
# Display media type occurences of the reduced and balance dataset
print_media_occ('Reduced Article DB', reduced_article_db)
    

Article DB
+------------+--------+
| Media Type | Count  |
+------------+--------+
| News       | 734488 |
| Blog       | 265512 |
+------------+--------+

Reduced Article DB
+------------+-------+
| Media Type | Count |
+------------+-------+
| News       | 10000 |
| Blog       | 10000 |
+------------+-------+


### Helper methods for feature extraction

In [6]:
lmtzr = nltk.stem.wordnet.WordNetLemmatizer()
sid = SentimentIntensityAnalyzer()

# Lemmatizing (the proper way, accounting for different POS tags)
def penn_to_wn(penn_tag):
    """
    Returns the corresponding WordNet POS tag for a Penn TreeBank POS tag.
    """
    if penn_tag in ['NN', 'NNS', 'NNP', 'NNPS']:
        wn_tag = wn.NOUN
    elif penn_tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        wn_tag = wn.VERB
    elif penn_tag in ['RB', 'RBR', 'RBS']:
        wn_tag = wn.ADV
    elif penn_tag in ['JJ', 'JJR', 'JJS']:
        wn_tag = wn.ADJ
    else:
        wn_tag = None
    return wn_tag

def get_lemmas(tokens):
    """Lemmatize the list of tokens"""
    tagged_tokens = nltk.pos_tag(tokens)
    lemmas = []
    for token, pos in tagged_tokens:
        wn_tag = penn_to_wn(pos)
        lemma = lmtzr.lemmatize(token) if not wn_tag else lmtzr.lemmatize(token, wn_tag)
        lemmas.append(lemma)
    return lemmas

def token_count_in_articles(articles):
    """
    Return occurence count of tokens in all articles.

    :param DataFrame articles: Collection of articles
    :returns int: A count value
    """
    total_count = defaultdict(int)
    for i in tqdm(range(len(articles)), desc='Counting token occurences'):
        text = articles.iloc[i]['content'].lower()
        tokens = nltk.word_tokenize(text)
        token_count = get_token_count(tokens)
        for token, count in token_count.items():
            total_count[token] += count
    return total_count

def get_token_count(tokens):
    """
    Count occurences of tokens in text.

    :param list tokens: List of tokens
    :returns dict: Dict of tokens and their count
    """
    token_counts = dict(Counter(tokens))
    return token_counts

def get_spelling_errors(text):
    """
    Count number of spelling errors in text.

    :param str text
    :returns int: Number of spelling errors
    """
    tokens = nltk.word_tokenize(text)
    cnt = 0
    for token in tokens:
        # If spelling error and token is not a stop word
        if not (wn.synsets(token) and token in stop_words_en):
            cnt += 1
    return cnt


### Display discrepancies of token occurences between media types

In [7]:
assert article_db is not None or reduced_article_db is not None, \
    "Load either the full article dataset or the reduced article \
    dataset before executing this cell."

if article_db is not None:
    # Randomly select 40 000 articles of each type and measure token occurence discrepancy
    idx = np.random.randint(0, 200000, size=40000)
    news_articles = article_db[article_db['media-type'] == 'News'].iloc[idx]
    blog_articles = article_db[article_db['media-type'] == 'Blog'].iloc[idx]
else:
    # Load reduced article dataset
    news_articles = reduced_article_db[reduced_article_db['blog'] == 0]
    blog_articles = reduced_article_db[reduced_article_db['blog'] == 1]
    
token_count_news = token_count_in_articles(news_articles)
token_count_blogs = token_count_in_articles(blog_articles)

# Get all tokens
tokens = set(token_count_news.keys())
tokens.update(list(token_count_blogs.keys()))

# Get the token occurence difference between news and blog articles
diff_count = defaultdict(int)
for token in tqdm(tokens, desc='Determine occurence diff.'):
    diff_count[token] += token_count_blogs.get(token, 0)
    diff_count[token] -= token_count_news.get(token, 0)

# Sort the tokens by the largest occurence difference
sorted_diff_count = sorted(diff_count.items(), key=lambda x: x[1], reverse=True)
first_hundred = sorted_diff_count[:100]     # Higher occurence in blogs than news
last_hundred = sorted_diff_count[-100::-1]  # Higher occurence in news than blogs

# Display the tokens with the largest difference 
# (idea is that the count of tokens with a large occurence diff will make good features)
print (AsciiTable([['Tokens with largest occurence diff. between media types']]).table)
print ("- Positive values indicate a higher occurence in blogs than news")
table_data = [['Num.', 'Token', 'Count Diff', 'Token', 'Count Diff']]
for i in range(len(first_hundred)):
    f_token, f_diff = first_hundred[i]
    l_token, l_diff = last_hundred[i]
    table_data.append([i, f_token, f_diff, l_token, l_diff])
print (AsciiTable(table_data).table)



Determine occurence diff.: 100%|██████████| 552835/552835 [00:01<00:00, 368255.17it/s]


+---------------------------------------------------------+
| Tokens with largest occurence diff. between media types |
+---------------------------------------------------------+
- Positive values indicate a higher occurence in blogs than news
+------+-----------+------------+---------------+------------+
| Num. | Token     | Count Diff | Token         | Count Diff |
+------+-----------+------------+---------------+------------+
| 0    | i         | 73575      | against       | -3137      |
| 1    | you       | 68140      | markets       | -3129      |
| 2    | ’         | 61844      | capital       | -3039      |
| 3    | !         | 37349      | national      | -3037      |
| 4    | :         | 35841      | source        | -3029      |
| 5    | it        | 34568      | %             | -2991      |
| 6    | your      | 30384      | announced     | -2988      |
| 7    | my        | 27277      | cent          | -2985      |
| 8    | s         | 26736      | according     | -2974      |

In [8]:
# Cherry-picked tokens whose occurence count will act as features
tokens_to_check = ['i', 
                   'you', 
                   'me', 
                   'we', 
                   'my', 
                   'this', 
                   'that', 
                   'it', 
                   'like',
                   '--', 
                   '-', 
                   "''", 
                   '%', 
                   'said',
                   'told',
                   '?', 
                   '!', 
                   '’', 
                   ':']

### Build feature set

In [13]:
assert reduced_article_db is not None, \
    "Load a reduced and balanced version of the original dataset\
     before executing this cell."
    
lmtzr = nltk.stem.wordnet.WordNetLemmatizer()
sid = SentimentIntensityAnalyzer()

feature_set = []
for i in tqdm(range(reduced_article_db.shape[0]), desc='Building feature set'):
    # Extract the article
    article = reduced_article_db.iloc[i]
    
    # Extract the article content
    article_text = article['content']
    
    # Extract tokens and sentences from the text
    tokens = nltk.word_tokenize(article_text.lower())
    sentences = nltk.sent_tokenize(article_text.lower())
    
    article_features = {}
    article_features['article_length'] = len(article_text)
    # Average token length
    article_features['token_length'] = sum(len(t) for t in tokens) / len(tokens)
    # Spelling errors per token
    article_features['spelling_errors'] = get_spelling_errors(article_text) / len(tokens)
    
    # Get token count in the article
    token_counts = get_token_count(tokens)

    # Token occurence features
    for token in tokens_to_check:
        article_features[token] = token_counts.get(token, 0)
      
    # Sentiment features, averaged per sentence
    sentiments = {'pos': 0, 'neg': 0, 'neu': 0, 'comp': 0}
    for sentence in sentences:
        sentiment = sid.polarity_scores(sentence)
        sentiments['pos'] += sentiment['pos']
        sentiments['neg'] += sentiment['neg']
        sentiments['neu'] += sentiment['neu']
        sentiments['comp'] += sentiment['compound']
    
    # Save the average sentiment scores
    article_features['sent_pos'] = sentiments['pos'] / len(sentences)
    article_features['sent_neg'] = sentiments['neg'] / len(sentences)
    article_features['sent_neu'] = sentiments['neu'] / len(sentences)
    article_features['sent_comp'] = sentiments['comp'] / len(sentences)
    
    # Target value
    article_features['blog'] = article['blog']
    
    feature_set.append(article_features)
    
dataset = pd.DataFrame(feature_set)

Building feature set: 100%|██████████| 20000/20000 [09:24<00:00, 35.40it/s]


### Save dataset

In [14]:
# Save dataset
dataset.to_pickle('%s/dataset' % PATH_TO_DATA)